In [1]:
# Import our dependencies
import pandas as pd
from path import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [2]:
# Import our clean dataset
file_path = Path("../clean_data/combined_olympic_data.csv")
df = pd.read_csv(file_path)
df.head(10)

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,total
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4
4,Armenia,2912403,3986.231624,0.733,0.299,37,2
5,Australia,23475686,62510.791170,0.935,0.123,80,38
6,Austria,8546356,51717.495940,0.885,0.083,72,17
7,Azerbaijan,9535079,7891.313147,0.751,0.330,29,9
8,Burundi,9844297,274.857948,0.400,0.483,20,0
9,Belgium,11209057,47700.540360,0.890,0.076,76,3


In [3]:
df["total"].value_counts()

0      75
1      18
2       8
3       6
4       5
6       5
8       4
17      3
5       3
9       3
12      3
13      3
7       2
15      2
18      2
38      2
100     1
19      1
16      1
21      1
23      1
31      1
36      1
43      1
44      1
46      1
50      1
63      1
70      1
97      1
132     1
Name: total, dtype: int64

In [4]:
bin_labels = ['0', '1-5', '6-10', '11-30', '31-50', '51+']
num_bins = len(bin_labels)

df['medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=bin_labels)
df['numerical_medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 30, 50, 140],
                              labels=range(num_bins))

In [5]:
df['medal_grouping'].value_counts()

0        75
1-5      40
11-30    17
6-10     14
31-50     8
51+       5
Name: medal_grouping, dtype: int64

In [6]:
new_df = df.rename(columns={"total": "count_of_medals"})
new_df.head()

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1,1-5,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0,0,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0,0,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4,1-5,1
4,Armenia,2912403,3986.231624,0.733,0.299,37,2,1-5,1


### Split our preprocessed data into our features and target arrays

In [7]:
# Create our target
y = new_df["numerical_medal_grouping"].values
y[:20]

[1, 0, 0, 1, 1, ..., 1, 0, 3, 0, 3]
Length: 20
Categories (6, int64): [0 < 1 < 2 < 3 < 4 < 5]

In [8]:
# Optimizing and transforming features
# Transforming CPI to values between 0-1
new_df["corruption_perceptions_index"] = new_df["corruption_perceptions_index"] / 100
# Transforming GII to invert values. Higher values will now correspond to more gender equality.
new_df["gender_inequality_index"] = 1 - new_df["gender_inequality_index"]
# Replacing GDP per capita with total GDP
new_df["gdp_total"] = new_df["population"] * new_df["gdp_per_capita"]
new_df.head()

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping,gdp_total
0,Afghanistan,33370794,613.856689,0.465,0.324,0.12,1,1-5,1,2.048489e+10
1,Albania,2889104,4578.631994,0.733,0.733,0.33,0,0,0,1.322814e+10
2,United Arab Emirates,9214175,43751.838890,0.835,0.753,0.70,0,0,0,4.031371e+11
3,Argentina,42669500,12334.798250,0.836,0.636,0.34,4,1-5,1,5.263197e+11
4,Armenia,2912403,3986.231624,0.733,0.701,0.37,2,1-5,1,1.160951e+10


In [9]:
# Create our features
X = new_df.drop(["country_name","count_of_medals", "medal_grouping", "numerical_medal_grouping"], axis=1).values
X

array([[3.33707940e+07, 6.13856689e+02, 4.65000000e-01, 3.24000000e-01,
        1.20000000e-01, 2.04848851e+10],
       [2.88910400e+06, 4.57863199e+03, 7.33000000e-01, 7.33000000e-01,
        3.30000000e-01, 1.32281440e+10],
       [9.21417500e+06, 4.37518389e+04, 8.35000000e-01, 7.53000000e-01,
        7.00000000e-01, 4.03137100e+11],
       [4.26695000e+07, 1.23347982e+04, 8.36000000e-01, 6.36000000e-01,
        3.40000000e-01, 5.26319674e+11],
       [2.91240300e+06, 3.98623162e+03, 7.33000000e-01, 7.01000000e-01,
        3.70000000e-01, 1.16095129e+10],
       [2.34756860e+07, 6.25107912e+04, 9.35000000e-01, 8.77000000e-01,
        8.00000000e-01, 1.46748371e+12],
       [8.54635600e+06, 5.17174959e+04, 8.85000000e-01, 9.17000000e-01,
        7.20000000e-01, 4.41996132e+11],
       [9.53507900e+06, 7.89131315e+03, 7.51000000e-01, 6.70000000e-01,
        2.90000000e-01, 7.52442943e+10],
       [9.84429700e+06, 2.74857948e+02, 4.00000000e-01, 5.17000000e-01,
        2.00000000e-01, 

In [10]:
# Splitting data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)
X_train.shape
X_test.shape
y_train.shape
y_test.shape
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape, new_df.shape)

(119, 6) (40, 6) (119,) (40,) (159, 10)


In [11]:
#Create a StandardScaler instances
scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [12]:
# Saving values for result comparison later on
y_test_values = y_test

In [13]:
# We need to convert our target labels (expected values) to categorical data
num_classes = num_bins
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# Original label of `1` is one-hot encoded as `0100000000`
y_train[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 3 * number_input_features
hidden_nodes_layer2 = 2 * number_input_features
hidden_nodes_layer3 = 2 * number_input_features

nn = Sequential()
number_input_features

6

In [15]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=num_classes, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 18)                126       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 78        
Total params: 588
Trainable params: 588
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, shuffle=True)

Train on 119 samples
Epoch 1/150
119/119 [==============================] - 1s 9ms/sample - loss: 1.8211 - accuracy: 0.0588
Epoch 2/150
119/119 [==============================] - 0s 136us/sample - loss: 1.8017 - accuracy: 0.1008
Epoch 3/150
119/119 [==============================] - 0s 136us/sample - loss: 1.7833 - accuracy: 0.0840
Epoch 4/150
119/119 [==============================] - 0s 151us/sample - loss: 1.7648 - accuracy: 0.1513
Epoch 5/150
119/119 [==============================] - 0s 183us/sample - loss: 1.7500 - accuracy: 0.2101
Epoch 6/150
119/119 [==============================] - 0s 170us/sample - loss: 1.7376 - accuracy: 0.3277
Epoch 7/150
119/119 [==============================] - 0s 110us/sample - loss: 1.7249 - accuracy: 0.3866
Epoch 8/150
119/119 [==============================] - 0s 148us/sample - loss: 1.7166 - accuracy: 0.3866
Epoch 9/150
119/119 [==============================] - 0s 123us/sample - loss: 1.7062 - accuracy: 0.4286
Epoch 10/150
119/119 [==============

119/119 [==============================] - 0s 158us/sample - loss: 1.2291 - accuracy: 0.4622
Epoch 79/150
119/119 [==============================] - 0s 101us/sample - loss: 1.2228 - accuracy: 0.4622
Epoch 80/150
119/119 [==============================] - 0s 151us/sample - loss: 1.2187 - accuracy: 0.4622
Epoch 81/150
119/119 [==============================] - 0s 160us/sample - loss: 1.2121 - accuracy: 0.4622
Epoch 82/150
119/119 [==============================] - 0s 180us/sample - loss: 1.2079 - accuracy: 0.4622
Epoch 83/150
119/119 [==============================] - 0s 183us/sample - loss: 1.2024 - accuracy: 0.4538
Epoch 84/150
119/119 [==============================] - 0s 132us/sample - loss: 1.1975 - accuracy: 0.4706
Epoch 85/150
119/119 [==============================] - 0s 127us/sample - loss: 1.1928 - accuracy: 0.4706
Epoch 86/150
119/119 [==============================] - 0s 119us/sample - loss: 1.1889 - accuracy: 0.4706
Epoch 87/150
119/119 [==============================] - 0s 

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

40/1 - 0s - loss: 1.0982 - accuracy: 0.5750
Loss: 1.170953059196472, Accuracy: 0.574999988079071


In [19]:
# Make predictions with scaled test data
y_test_pred = nn.predict(X_test_scaled)

In [20]:
y_test

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1.

In [21]:
y_test_pred.round(1)

array([[0.8, 0.2, 0. , 0. , 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.2, 0. , 0. ],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.8, 0.1, 0. , 0. , 0. , 0. ],
       [0.3, 0.5, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.1, 0.1, 0.1, 0.2, 0.3, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.6, 0.3, 0.1, 0. , 0. , 0. ],
       [0.3, 0.5, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.5, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.4, 0.4, 0.1, 0.1, 0. , 0. ],
       [0.7, 0.3, 0. , 0. , 0. , 0. ],
       [0.2, 0.5, 0.1, 0.1, 0. , 0. ],
       [0.6, 0.3, 0.1, 0.1, 0. , 0. ],
       [0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0. , 0. , 0. , 0.1, 0.4, 0.5],
       [0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.7, 0.2, 0. , 0. , 0. , 0. ],
       [0.2, 0.4, 0.2, 0.1, 0. , 0. ],
       [0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0.5, 0.4, 0.1, 0.

In [22]:
nn.predict_classes(X_test_scaled)

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 4, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 5, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 4, 4, 0, 1])

In [23]:
results_df = pd.DataFrame(y_test_values)
results_df['model_results'] = nn.predict_classes(X_test_scaled)

results_df

,0,model_results
0,0,0
1,3,0
2,1,0
3,0,0
4,1,1
5,0,0
6,0,0
7,1,1
8,0,0
9,3,4


### Saving the model

In [24]:
# Export our model to HDF5 file
nn.save("nn_trained_categories_model_.h5")

In [25]:
# new_df.reset_index(inplace=True)
new_results_df = new_df.join(results_df).head(50)
new_results_df

# pd.concat([new_df, results_df], axis=1)

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping,gdp_total,0,model_results
0,Afghanistan,33370794,613.856689,0.465,0.3240,0.12,1,1-5,1,2.048489e+10,0,0.0
1,Albania,2889104,4578.631994,0.733,0.7330,0.33,0,0,0,1.322814e+10,3,0.0
2,United Arab Emirates,9214175,43751.838890,0.835,0.7530,0.70,0,0,0,4.031371e+11,1,0.0
3,Argentina,42669500,12334.798250,0.836,0.6360,0.34,4,1-5,1,5.263197e+11,0,0.0
4,Armenia,2912403,3986.231624,0.733,0.7010,0.37,2,1-5,1,1.160951e+10,1,1.0
5,Australia,23475686,62510.791170,0.935,0.8770,0.80,38,31-50,4,1.467484e+12,0,0.0
6,Austria,8546356,51717.495940,0.885,0.9170,0.72,17,11-30,3,4.419961e+11,0,0.0
7,Azerbaijan,9535079,7891.313147,0.751,0.6700,0.29,9,6-10,2,7.524429e+10,1,1.0
8,Burundi,9844297,274.857948,0.400,0.5170,0.20,0,0,0,2.705783e+09,0,0.0
9,Belgium,11209057,47700.540360,0.890,0.9240,0.76,3,1-5,1,5.346781e+11,3,4.0


In [26]:
new_results_df = new_results_df.dropna()

In [27]:
# new_results_df.drop(["population","gdp_per_capita","human_development_index","gender_inequality_index","corruption_perceptions_index","gdp_total",], axis=1)

new_results_df = new_results_df[["country_name",0,"model_results"]]

In [28]:
new_results_df.set_index("country_name",drop=True)

,0,model_results
country_name,,
Afghanistan,0,0.0
Albania,3,0.0
United Arab Emirates,1,0.0
Argentina,0,0.0
Armenia,1,1.0
Australia,0,0.0
Austria,0,0.0
Azerbaijan,1,1.0
Burundi,0,0.0


In [29]:
new_results_df.to_csv("nn_model_results.csv")